In [ ]:
import pandas as pd
#Loading of data
df = pd.read_csv('LeetCode_Questions.csv.csv')

#Renaming of columns to our needs
df = df.rename(columns={
    'Question Text': 'description',
    'Question Title': 'title',
    'Difficulty Level': 'problem_class'
})

#Filling missing info.
df['input_description'] = "See problem description."
df['output_description'] = "See problem description."

#Creating score for our data
difficulty_map = {'Easy': 10, 'Medium': 30, 'Hard': 60}
df['problem_score'] = df['problem_class'].map(difficulty_map)

#Save
df = df[['title', 'description', 'input_description', 'output_description', 'problem_class', 'problem_score']]
df.to_csv('final_dataset.csv', index=False)

print("Data is ready! Here is a preview:")
display(df.head())

Data is ready! Here is a preview:


,title,description,input_description,output_description,problem_class,problem_score
0,Two Sum,Given an array of integers nums and an integer...,See problem description.,See problem description.,Easy,10
1,Add Two Numbers,You are given two non-empty linked lists repre...,See problem description.,See problem description.,Medium,30
2,Longest Substring Without Repeating Characters,"Given a string s, find the length of the longe...",See problem description.,See problem description.,Medium,30
3,Median of Two Sorted Arrays,Given two sorted arrays nums1 and nums2 of siz...,See problem description.,See problem description.,Hard,60
4,Longest Palindromic Substring,"Given a string s, return the longest palindrom...",See problem description.,See problem description.,Medium,30


In [ ]:
import pandas as pd
import re
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

#Loading dataset
df = pd.read_csv('final_dataset.csv')

#Cleaning text
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return text

#Combining columns
df['full_text'] = df['title'].astype(str) + " " + df['description'].astype(str) + " " + df['input_description'].astype(str)
df['clean_text'] = df['full_text'].apply(clean_text)

#Extraction
vectorizer=TfidfVectorizer(max_features=1000)
X=vectorizer.fit_transform(df['clean_text'])
y_class=df['problem_class']
y_score=df['problem_score']

#Model_1 (Classifier)
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X, y_class)

#Model_2 (Regressor)
reg = RandomForestRegressor(n_estimators=100, random_state=42)
reg.fit(X, y_score)

#Saving
joblib.dump(clf, 'model_classifier.pkl')
joblib.dump(reg, 'model_regressor.pkl')
joblib.dump(vectorizer, 'vectorizer.pkl')

print("Models trained and saved successfully.")

Models trained and saved successfully.


In [ ]:
#Creating app.py
%%writefile app.py
import streamlit as st
import joblib
import re
import numpy as np

#Uploading our models
try:
    classifier = joblib.load('model_classifier.pkl')
    regressor = joblib.load('model_regressor.pkl')
    vectorizer = joblib.load('vectorizer.pkl')
except:
    st.error("Error: Model files not found! Make sure you ran the training code first.")
    st.stop()

#Cleaning the text
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return text

#Layout
st.title("AutoJudge: AI Problem Evaluator")
st.write("Paste your coding problem below to predict its Difficulty and Score.")

#Input/Output boxes
title = st.text_input("Problem Title", "Example: Sum of Array")
desc = st.text_area("Problem Description", "Given an array of integers, find the sum of its elements.")
inp_desc = st.text_area("Input Description", "The first line contains N.")
out_desc = st.text_area("Output Description", "Print the sum.")

if st.button("Predict Difficulty"):
    #Combining inputs
    full_text = f"{title} {desc} {inp_desc} {out_desc}"

    #Cleaning and transforming into numerical
    cleaned = clean_text(full_text)
    vectorized_text = vectorizer.transform([cleaned])

    #Prediction
    pred_class = classifier.predict(vectorized_text)[0]
    pred_score = regressor.predict(vectorized_text)[0]

    #Result
    st.success("Prediction Complete!")

    col1, col2 = st.columns(2)
    with col1:
        st.metric("Difficulty Class", pred_class)
    with col2:
        st.metric("Difficulty Score", f"{pred_score:.1f}")

    st.progress(int(min(max(pred_score, 0), 100)))

print("Website code created successfully!")

Writing app.py


In [18]:
!pip install -q streamlit pyngrok
from pyngrok import ngrok
ngrok.set_auth_token("37DciWk2Ul6kUUK266QTeZwARVw_84cSK3B4GT6e16Do451rH")
!nohup streamlit run app.py --server.port 8501 &

public_url = ngrok.connect(8501).public_url
print(f"CLICK THIS SAFE LINK: {public_url}")

nohup: appending output to 'nohup.out'
CLICK THIS SAFE LINK: https://orientally-cannular-jona.ngrok-free.dev


In [19]:
import subprocess
import time
!pkill -9 -f streamlit
!pkill -9 -f localtunnel
!pkill -9 -f cloudflared
!pip install -q streamlit
#cloudflare
!wget -q -O cloudflared-linux-amd64 https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
print("⏳ Starting Streamlit server...")
subprocess.Popen(["streamlit", "run", "app.py", "--server.headless=true", "--theme.base=light"])
print("⏳ Opening secure tunnel...")
with open("tunnel.log", "w") as f:
    subprocess.Popen(["./cloudflared-linux-amd64", "tunnel", "--url", "http://localhost:8501"], stdout=f, stderr=f)

time.sleep(8)
print("DONE! Click the link below (No password needed):")
!grep -o 'https://.*\.trycloudflare.com' tunnel.log

⏳ Starting Streamlit server...
⏳ Opening secure tunnel...


DONE! Click the link below (No password needed):
https://part-loved-finding-its.trycloudflare.com
